In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import scienceplots
plt.style.use(['science','notebook','grid'])

In [3]:
Av = 15.71511 #MeV
As = 17.53638 #MeV
Ac = 0.71363 #MeV
Aa = 23.37837 #MeV

In [18]:
def y(x):
    return np.sqrt(As/(2*Ac*x*x*x))

In [4]:
def f(x):
    return 0.5/(1 + Ac*x*x/(4*Aa)) - np.sqrt(As/(2*Ac*x*x*x))

In [17]:
sol = root(f,3)
xstable = sol['x'][0]
ystable = y(xstable)

In [34]:
Astable = xstable**3
Zstable = ystable*Astable
(Zstable,Astable)

(27.496349566027305, 61.53372802754275)

In [36]:
print(f'The most stable element is the one with Z = {int(Zstable)} and A = {int(Astable)}.')

The most stable element is the one with Z = 27 and A = 61.
